## 1. THU THẬP DỮ LIỆU

### Import thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import requests
import re
import os

### Thiết lập trình duyệt

In [2]:
options = Options()
# options.add_argument("--headless") # Chạy không hiển thị trình duyệt
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

### Khởi tạo trình duyệt

In [3]:
service = Service("../data/metadata/chromedriver.exe")  # Tải trình duyệt ChromeDriver tương ứng (ở đây tôi sử dụng ChromeDriver 132.0.6834.159)
driver = webdriver.Chrome(service=service, options=options)

In [4]:
save_dir = "../data/raw_images"
os.makedirs(save_dir, exist_ok=True)

In [5]:
def norm_text(text):
    text = text.lower()
    text = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', text)
    text = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', text)
    text = re.sub(r'[ìíịỉĩ]', 'i', text)
    text = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', text)
    text = re.sub(r'[ùúụủũưừứựửữ]', 'u', text)
    text = re.sub(r'[ỳýỵỷỹ]', 'y', text)
    text = re.sub(r'đ', 'd', text)
    text = re.sub(r'[^a-z0-9]', '_', text)
    return text

In [6]:
def food_crawler(food_names):
    failed_foods = []
    
    for food_name in food_names:
        url_food_name = food_name.replace(" ", "%20") # %20 là ký tự khoảng trắng trong URL
        url = f"https://www.pinterest.com/search/pins/?q={url_food_name}"

        try:
            # Mở trang web
            driver.get(url)
            time.sleep(5)  # Chờ trang tải
            
            # Lấy danh sách ảnh
            images = driver.find_elements(By.TAG_NAME, "img")
            file_food_name = norm_text(food_name)

            counter = 1
            for img in images:
                if counter > 10:  # Giới hạn 10 ảnh
                    break
                
                # Lọc ảnh nhỏ (ví dụ icon 60x60)
                try:
                    width = int(img.get_attribute("width") or 0)
                    height = int(img.get_attribute("height") or 0)
                    if width < 150 or height < 150:
                        continue  # Bỏ qua ảnh nhỏ
                except:
                    continue  # Nếu không lấy được kích thước, bỏ qua

                # Lấy ảnh từ src hoặc srcset
                img_url = img.get_attribute("srcset") or img.get_attribute("src")
                if img_url:
                    img_url = img_url.split(",")[-1].split(" ")[0]  # Lấy ảnh lớn nhất trong srcset
                    if not (".jpg" in img_url or ".png" in img_url):
                        continue  # Bỏ qua ảnh không hợp lệ

                    ext = "jpg" if "jpg" in img_url else "png"
                    img_name = f"{file_food_name}_{counter}.{ext}"
                    img_path = os.path.join(save_dir, img_name)

                    try:
                        img_data = requests.get(img_url).content
                        with open(img_path, "wb") as f:
                            f.write(img_data)
                        print(f"✅ Đã tải: {img_path} ({width}x{height})")
                        counter += 1
                    except Exception as e:
                        print(f"⚠️ Lỗi khi tải ảnh {img_url}: {str(e)}")

        except Exception as e:
            print(f"⚠️ Lỗi khi xử lý món {food_name}: {str(e)}")
            failed_foods.append(food_name)

    driver.quit()
    print("🎉 Hoàn thành!")
    return failed_foods

### Main

In [7]:
with open("../data/metadata/food_cate.txt", "r", encoding="utf-8") as file:
    food_list = [line.strip() for line in file.readlines()]
food_list[:5]

['Phở', 'Bún chả', 'Bánh cuốn', 'Chả cá Lã Vọng', 'Bún đậu mắm tôm']

In [8]:
failed_foods = food_crawler(food_list) # Lưu lại danh sách món ăn không tải được (đồng thời chạy crawl ảnh tự động)

✅ Đã tải: ../data/raw_images\pho_1.jpg (231x410)
✅ Đã tải: ../data/raw_images\pho_2.jpg (231x346)
✅ Đã tải: ../data/raw_images\pho_3.jpg (231x346)
✅ Đã tải: ../data/raw_images\pho_4.jpg (231x231)
✅ Đã tải: ../data/raw_images\pho_5.jpg (231x346)
✅ Đã tải: ../data/raw_images\pho_6.jpg (231x231)
✅ Đã tải: ../data/raw_images\pho_7.jpg (231x306)
✅ Đã tải: ../data/raw_images\pho_8.jpg (231x308)
✅ Đã tải: ../data/raw_images\pho_9.jpg (231x307)
✅ Đã tải: ../data/raw_images\bun_cha_1.jpg (235x353)
✅ Đã tải: ../data/raw_images\bun_cha_2.jpg (235x278)
✅ Đã tải: ../data/raw_images\bun_cha_3.jpg (235x156)
✅ Đã tải: ../data/raw_images\bun_cha_4.jpg (235x313)
✅ Đã tải: ../data/raw_images\bun_cha_5.jpg (235x329)
✅ Đã tải: ../data/raw_images\bun_cha_6.jpg (235x235)
✅ Đã tải: ../data/raw_images\bun_cha_7.jpg (235x348)
✅ Đã tải: ../data/raw_images\bun_cha_8.jpg (235x313)
✅ Đã tải: ../data/raw_images\bun_cha_9.jpg (235x353)
✅ Đã tải: ../data/raw_images\banh_cuon_1.jpg (235x313)
✅ Đã tải: ../data/raw_image

In [9]:
print("Danh sách món ăn không tải được:")
for food in failed_foods:
    print(food)

Danh sách món ăn không tải được:
